# 21. 클래스 메타프로그래밍

## Index
- 21.1 클래스 팩토리
- 21.2 디스크립터를 커스터마이즈하기 위한 클래스 데커레이터
- 21.3 임포트 타임과 런타임
- 21.4 메타클래스 기본 지식
- 21.5 디스크립터를 커스터마이즈하기 위한 메타클래스
- 21.6 메타클래스 \__prepare\__() 특별 메서드
- 21.7 객체로서의 클래스

<blockquote>meta는 영어의 접두사로, 다른 개념으로부터의 추상화를 가리키며 후자를 완성하거나 추가하는 데에 쓰인다.</blockquote>

클래스 메타프로그래밍은 런타임에 클래스를 만들거나 커스터마이징 하는 기술이다. 파이썬에서 클래스는 일급 객체이다. 따라서 클래스를 만드는 또 다른 클래스가 있을 수 있다. 이것을 metaclass라고 한다.

## 21.1 클래스 팩토리

In [19]:
#Example 21-1
Dog = record_factory('Dog','name weight owner')
rex = Dog('Rex', 30, 'Bob')
rex

Dog(name='Rex', weight=30, owner='Bob')

In [20]:
name, weight, _ = rex
name, weight

('Rex', 30)

In [21]:
"{2}'s dog weighs {1}kg".format(*rex)

"Bob's dog weighs 30kg"

In [22]:
rex.weight = 32
rex

Dog(name='Rex', weight=32, owner='Bob')

In [23]:
Dog(name='Rex', weight=32, owner='Bob')
Dog.__mro__

(__main__.Dog, object)

In [13]:
#Example 21-2

def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split()
    except AttributeError: #no .replace or .split
        pass # assume it's already a sequence of identifiers
    field_names = tuple(field_names)
    
    def __init__(self, *args, **kwargs):
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self):
        for name in self.__slots__:
            yield getattr(self, name)
            
    def __repr__(self):
        values = ', '.join('{}={!r}'.format(*i) for i
                          in zip(self.__slots__, self))
        return '{}({})'.format(self.__class__.__name__, values)
    
    cls_attrs = dict(__slots__ = field_names,
                     __init__ = __init__,
                     __iter__ = __iter__,
                     __repr__ = __repr__)
    
    return type(cls_name, (object, ), cls_attrs)

`type(my_object)`는 함수가 아니라 클래스이다.
``` Python
MyClass = type('MyClass', (MySuperClass, MyMixin),
               {'x':42, 'x2': lambda self: self.x * 2})
```
는
``` Python
class MyClass(MySuperClass, MyMixin):
    x = 42
    
    def x2(self):
        return self.x * 2
```
`record_factory`로 만든 인스턴스는 직렬화할 수 없다. 따라서 `pickle`을 사용할 수 없다. 즉 `type`의 인스턴스가 `MyClass`와 같이 클래스라는 것. 

## 21.2 디스크립터를 커스터마이즈하기 위한 클래스 데커레이터
20절의 LineItem Take #5에서 저장소 이름이 `_Quantity#0`과 같이 저장되어 디버깅하기 어려운 문제가 있었다. 
```Python
>>> LineItem.weight.storage_name
'_Quantity#0'
```
으로 나오는 것보다, 실제 이름이 포함되어
``` Python
'_Quantity#weight'
```
로 나오는 것이 좋다.

Lineitem Take #4를 기억해보면, 디스크립터가 인스턴스화 되었을 때는 관리 속성의 이름을 알 수 있는 방법이 없기 때문에 descriptive storage name을 사용하지 않는다. 하지만 일단 전체 클래스가 바운딩 되어서 완성되면, 검사도 할 수 있고 디스크립터에 이름도 지을 수 있다. 이것은 LineItem 클래스의 `__new__` 메서드에서 수행 할 수 있으므로 디스크립터가 `__init__` 메서드에서 사용될 때까지 올바른 storage name이 설정된다. `__new__`의 문제는 모든 새로운 `LineItem` 인스턴스를 만들 때마다 호출된다는 것. 디스크립터의 바인딩은 한번만 하면 되는데! 클래스가 만들어질 때 이름을 정해보자. 이것은 '클래스 데커레이터'나 '메타클래스'를 사용하면 할 수 있다.

In [14]:
#Example 21-3

import model_v6 as model

@model.entity
class LineItem:
    description = model.NonBlank()
    weight = model.Quantity()
    price = model.Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
            
    def subtotal(self):
        return self.weight * self.price

ModuleNotFoundError: No module named 'model_v6'

인터프리터가 평가하고 결과 클래스 객체는 `model.entity`에 전달. Python은 `model.entity`가 반환하는 것을 글로벌 이름 `LineItem`에 바인딩.

In [15]:
#Example 21-4: a class decorator

def entity(cls):
    for key, attr in cls.__dict__.items():
        if isinstance(attr, Validated):
            #if the attribute is one of our Validated descriptors
            type_name = type(attr).__name__
            attr.storage_naem = '_{}#{}'.format(type_name, key)
            #naming convention
    return cls

클래스 데커레이터의 큰 단점은 직접적으로 적용되는 클래스에서만 작동한다는 것이다. 이것은 데코레이터가 변경한 내용에 따라 데코레이터가 변경한 내용을 데코 레이팅된 클래스의 서브 클래스가 상속할 수도 있고 상속하지 않을 수도 있음을 의미함.

## 21.3 임포트 타임과 런타임
중간에 애매한 곳이 있어서 정확히 나누기는 힘들다.
### import time
인터프리터가 소스코드를 읽어서 실행가능한 바이트코드를 만들어낸다. `syntax error`도 이 때 일어남. local `__pycache__`에 최신의 `.pyc` 파일이 있으면 이 과정은 건너뛴다.

### import문
단순한 선언이 아니라 프로세스에서 처음으로 가져올 때 가져온 모듈의 top-level code를 실제로 실행하는데 이 top-level code는 Runtime의 일반적인 작업을 포함하여 모든 작업을 수행할 수 있다. 그래서 정확히 import time과 runtime을 구분하기 어렵다! `import`는 모든 종류의 runtime 동작을 트리거 할 수 있다.

일반적인 경우, 인터프리터가 임포트 시 최상위 함수를 정의하지만 함수가 런타임에 호출되는 경우에만 본문을 실행함. 함수 본문을 컴파일하지만, 실행하지는 않음.

클래스는? 인터프리터가 클래스가 다른 클래스로 중첩되더라도 모든 클래스 본문을 다 실행함. 클래스 본문 실행의 의미는 속성과 함수가 정의되고 나서 클래스 객체 자체가 빌드되는 것을 의미함. 이러한 점에서, 클래스의 본문은 최상위 레벨 코드이고 import time에 실행된다.

이해가 안간다면?! `The Evaluation Time Exercise` 예제를 따라해보자!

## 21.4 메타클래스 기본 지식
메타클래스는 class factory이다. 클래스도 객체이므로 각각의 클래스는 다른 클래스의 객체가 되어야 한다. 기본적으로 클래스는 `type`의 인스턴스이다. 다른 말로 `type`은 대부분의 빌트인, 사용자 정의 클래스들의 메타클래스이다.

![fig21_2](../images/fig21_2.png)

- `object`는 `type`의 인스턴스이고, `type`은 `object`의 서브클래스이다.
- `type`은 `type` 그 자체의 인스턴스이다.
- 모든 클래스는 `type`의 인스턴스이지만, 모든 메타클래스는 또한 `type`의 서브클래스이기 때문에 class factory의 역할을 할 수 있다.
- 메타클래스는 `__init__`을 구현함으로 인스턴스를 커스터마이징 할 수 있다. `__init__` 메소드는 클래스 데커레이터가 하는 모든 것을 할 수 있지만 더 효과가 크다. 

`The Metaclass Evaluation Time Exercise`를 보자!

## 21.5 디스크립터를 커스터마이즈하기 위한 메타클래스
User-level을 더 간단하게 만들어보자!

In [16]:
#Example 21-14

import model_v7 as model

class LineItem(model.Entity):
    description = model.NonBlank()
    weight = model.Quantity()
    price = model.Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

ModuleNotFoundError: No module named 'model_v7'

In [24]:
class EntityMeta(type):
    """Metaclass for business entities with validated fields"""
    
    def __init__(cls, name, bases, attr_dict):
        super().__init__(name, bases, attr_dict)
        for key, attr in attr_dict.items():
            if isinstance(attr, Validated):
                type_name = type(attr).__name__
                attr.storage_name = '_{}#{}'.format(type_name, key)
                
class Entity(metaclass=EntityMeta):
    """Business entity with validated fields"""

NameError: name 'Validated' is not defined

![fig21_4](../images/fig21_4.png)
디스크립터(weight, price)의 설정은 `EntityMeta.__init__`에 의해서 실행된다.

## 21.6 메타클래스 \__prepare\__() 특별 메서드
기본적으로 이름-속성 매핑은 `dict`을 사용하기 때문에 순서 정보가 손실. `__prepare()__` 특별 메서드를 사용!

In [25]:
#Example 21-16

import collections

class EntityMeta(type):
    """Metaclass for business entities with validated fields"""
    
    @classmethod
    def __prepare__(cls, name, bases):
        return collections.OrderedDict() 
        #class 속성이 저장될 빈 OrderedDict
    
    def __init__(cls, name, bases, attr_dict):
        super().__init__(name, bases, attr_dict)
        cls._field_names = []
        for key, attr in attr_dict.items():
            #attr_dict은 __init__을 실행하기 전에 실행한 __prepare__에서 받아온
            #OrderedDict임. 따라서 추가된 순서대로 속성을 검토함.
            if isinstance(attr, Validated):
                type_name = type(attr).__name__
                attr.storage_name = '_{}#{}'.format(type_name, key)
                cls._field_names.append(key)
             
            
class Entity(metaclass=EntityMeta):
    """Business entity with validated fields"""
    
    @classmethod
    def field_names(cls):
        for name in cls._field_names:
            yield name

NameError: name 'Validated' is not defined

## 21.7 객체로서의 클래스
모든 클래스는 파이썬 데이터 모델에 정의된 많은 속성들이 있다. 지금까지 봤던 것들은 `__mro__`, `__class__`, `__name__`이 있고 다른 속성들도 알아보자.
### cls.\_\_bases\_\_
클래스의 베이스 클래스들의 튜플

### cls.\_\_qualname\_\_
모듈의 전역 범위에서 클래스 정의까지의 점선 경로(dotted path)인 클래스 또는 함수의 정규화 된 이름을 포함하는 Python 3.3의 새 속성. 예를 들어, 예제 21-6에서 inner class 'ClassTwo'의 `__qualname__`은 'ClassOne.ClassTwo'이고, `__name__`은 'ClassTwo'이다.

### cls.\__subclasses\__()
클래스의 직속 서브클래스들을 리스트로 반환하는 메소드. 순환 참조를 막기 위해 약한 참조를 사용하고 현재 메모리에 있는 서브클래스들의 목록을 반환한다.

### cls.mro()
인터프리터는 클래스의 \_\_mro\_\_ 속성에 저장된 슈퍼 클래스의 튜플을 얻기 위해 클래스를 빌드할 때 사용한다.

## 21.8 Reference
[핵심만 간단히, Hello world 파이썬 3!](https://wikidocs.net/21056)  
[알파해커 블로그](https://alphahackerhan.tistory.com/34)